### POLICY EVAULATION WITH MONTECARLO ###

In [1]:
import numpy as np 

def map_to_state(grid_position, grid_size)-> int:
    return grid_position[0] * grid_size + grid_position[1]   

def state_to_map(state, grid_size):
    i, j = divmod(state, grid_size)
    return [i,j]

def action_to_direction(action):
    direction = {
            0: np.array([1,0]),      # Giu
            1: np.array([0, 1]),     # Destra
            2: np.array([-1, 0]),    # Su
            3: np.array([0, -1]),    # Sinistra
        }   
    return direction[action]
    
def find_neighbors_states(cell, grid_size, actions):
    neighbors = []
    states = []
    for a in actions:
        next_cell = cell + action_to_direction(a)
        if valid_cell(next_cell, grid_size):
            neighbors.append(next_cell)
        else:
            neighbors.append(cell)

    for cell in neighbors:
        states.append(map_to_state(cell, grid_size))
    return states
        
def valid_cell(cell, grid_size) -> bool:
    if cell[0] < 0 or cell[1] < 0:
        return False
    elif cell[0] < grid_size and cell[1] < grid_size:
        return True
    else:
        return False

def neigh_to_action(curr_cell, neigh_cell):
    action = []
    if curr_cell != neigh_cell:
        
        delta_i = curr_cell[0] - neigh_cell[0]
        delta_j = curr_cell[1] - neigh_cell[1]

        if delta_i == 1:
            action = 2
        elif delta_i == -1:
            action = 0
        elif delta_j == 1:
            action = 3
        elif delta_j == -1:
            action = 1
            
    return action
            

In [3]:
grid_size = 4
states = np.array([], dtype=int)

# Trasformo tutte le posizioni della griglia in stati
for i in range(grid_size):
    for j in range(grid_size):
        states = np.append(states, map_to_state([i,j], grid_size))

V = np.full(states.shape, 0)
R = -1.0 * np.ones(len(states))

# Elenco possibili azioni
actions = [0, 1, 2, 3]

# Policy associata a ogni stato
Policy = {}
for state in states:
    Policy[state] = 0.25*np.ones(4)
    
# Probabilità di transizione associate alle azioni
Pa = {}
for k in actions:
    Pa[k] = np.zeros((len(states), len(states)))

    for i in range(grid_size):
        for j in range(grid_size):
            curr_cell = [i,j]
            curr_state = map_to_state(curr_cell, grid_size)
            next_cell = curr_cell + action_to_direction(k)

            # Se la cella è nella gridmap, la probabilità di spostarmi in quella cella è 1
            if valid_cell(next_cell, grid_size):
                next_state = map_to_state(next_cell, grid_size)
                Pa[k][curr_state, next_state] = 1
                
            # Se la cella è fuori dalla gridmap, resto nella cella corrente con probabilità 1
            else:
                Pa[k][curr_state, curr_state] = 1

# Imposto la posizione dei goal. 
# Vengono modellati come stati dai quali non posso uscire e che non danno nessuna reward negativa ad ogni step
# goals_pos = [[0,0],[grid_size-1, grid_size-1]]
# goals_states = [map_to_state(goals_pos[0], grid_size), map_to_state(goals_pos[1], grid_size)]
goals_pos = [grid_size-1, grid_size-1]
goals_states = [map_to_state(goals_pos, grid_size)]

for state in goals_states:
    R[state] = 0
    # Quando mi trovo nel goal, ogni azione che eseguo mi lascia nel goal con p = 1
    for k in actions:
        Pa[k][state,:] = 0
        Pa[k][state,state] = 1


episodes_per_step = 10000


# Trasformo il Markov Decision Process in un Markov Reward Process
Rpi = np.full(R.shape, 0)
Ppi = np.zeros((len(states), len(states)))

for k in actions:
    # Vettore delle probabilità di selezionare l'azione k per ogni stato
    action_prob = np.array([Policy[state][k] for state in states])

    # Reward associate alla politica pi
    Rpi = action_prob * R + Rpi
    
    # Matrice di transizione associata alla politica Ppi
    Ppi = np.dot(np.diag(action_prob), Pa[k]) + Ppi

# Update Value function 
for _ in range(episodes_per_step):
    V = Rpi + np.dot(Ppi, V)

V_matrix = V.reshape((grid_size, grid_size))
print(V_matrix)


[[-59.42857143 -57.42857143 -54.28571429 -51.71428571]
 [-57.42857143 -54.57142857 -49.71428571 -45.14285714]
 [-54.28571429 -49.71428571 -40.85714286 -30.        ]
 [-51.71428571 -45.14285714 -30.           0.        ]]
